In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [7]:
from matplotlib import pyplot as plt
import sys
import matplotlib.animation as animation
import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats
import scipy.spatial.distance as dist
import numpy as np
import h5py
from tqdm import tqdm
import pandas as pd
import imageio

#import tables
#import deepdish as dd

sys.path.append('/home/jma819/post_cmfe_analysis')
import python_utils_jjm as utils_jjm

In [8]:
results_file = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/29-Mar_21_42_20_out.mat'

In [9]:
results_file.split('/')[-1]

'29-Mar_21_42_20_out.mat'

In [10]:
results_file[0:-4]+'_demixed_.h5'

'/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/29-Mar_21_42_20_out_demixed_.h5'

In [11]:
results = sio.loadmat(results_file)

##A is spatial contours

In [12]:
#img_file = '/Users/johnmarshall/Documents/Analysis/MiniscopeMovies/GRIN011/H10_M19_S59msCam1.tif'
#img_from_file = plt.imread(img_file)    
#d1, d2 = np.shape(img_from_file)
d1, d2 = 480, 752

In [13]:
dense_A = results['A'].todense()

In [14]:
frames = np.shape(results['C'])[0]

In [15]:
cells = np.shape(results['C'])[0]

In [17]:
#reshape for all frames
frame_range = (100, 150)
cells_reshaped = np.empty((cells, frame_range[1]-frame_range[0], d1, d2))
for cell in tqdm(range(cells)):
    A_reshaped = np.reshape(dense_A[:, cell], (d1, d2))
    cells_reshaped[cell] = np.array([np.dot(A_reshaped, results['C'][cell, frame]) for frame in range(frame_range[0], frame_range[1])])
    


100%|██████████| 148/148 [00:42<00:00,  3.64it/s]


In [18]:
hf = h5py.File(results_file[0:-4]+'_demixed_.h5', 'w')

In [19]:
hf.create_dataset('demixed_movie_array', data=cells_reshaped )

<HDF5 dataset "demixed_movie_array": shape (148, 50, 480, 752), type "<f8">

In [20]:
hf.close()

In [21]:
cells_recombined = np.sum(cells_reshaped, axis=0)

In [23]:
imageio.mimwrite(results_file[0:-4]+'_demixed_.mp4', cells_recombined, fps = 20)

Lossy conversion from float64 to uint8. Range [-0.3798849567984799, 12.091775452771081]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 11.713751442333251]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.8520886023244922, 11.34831881789857]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.2384764332364273, 11.557402771284801]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-2.0497256521789264, 11.439661809377398]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 11.32280260335843]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7103066350561602, 10.968474789416469]. Convert image to uint

In [5]:
## recombine demixed video with behavior data 
cnmfe_range = (0, 150)
dir_path = '/Volumes/My_Passport/cnmfe_analysis_files/GRIN011/'
frame_clock_info = 'timestamp.dat'
frame_clock_df = pd.read_table(dir_path+frame_clock_info)

# load time stamps 
msCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 0].set_index('frameNum')[cnmfe_range[0]:cnmfe_range[1]]
behavCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 1].set_index('frameNum')
# reset initial clock value to 0 
msCam_timestamps['sysClock'][1] = 0
behavCam_timestamps['sysClock'][1] = 0

/Users/johnmarshall/miniconda3/envs/caiman/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


In [7]:
# create df to find matched behavior cam frames
msCam_timestamps = jjm_utils.align_behavior_data(msCam_timestamps, behavCam_timestamps)
msCam_timestamps.reset_index(inplace=True)
msCam_timestamps.head()

100%|██████████| 150/150 [00:01<00:00, 89.27it/s]


,frameNum,camNum,sysClock,buffer,behavCam_frames,sys_clock_behavCam
0,1,0,0,1,1,0
1,2,0,7,1,1,0
2,3,0,58,1,3,47
3,4,0,108,1,5,111
4,5,0,158,1,6,143


In [9]:
# 
reader = imageio.get_reader('/Volumes/My_Passport/cnmfe_analysis_files/GRIN011/behavCam1.avi')

In [18]:
bd1, bd2 = reader.get_meta_data()['size']

In [28]:
1 in msCam_timestamps['behavCam_frames'].values

True

In [35]:
behavior_adjusted = np.empty((cnmfe_range[1], bd2, bd1, 3))    
#for frame in range(cnmfe_range[0], cnmfe_range[1])
frame = 0
for i, im in enumerate(reader):
    if 1 in msCam_timestamps['behavCam_frames'].values:
        behavior_adjusted[0] = im
        

In [36]:
# combine those matched frames into a video

imageio.mimwrite('/Volumes/My_Passport/cnmfe_analysis_files/GRIN011/behavior_adjusted.mp4', behavior_adjusted, fps = 20)